## AHP in Python

# The Comparissons



1. Get the data into Python

In [1]:
# the link to the data

linkGoogle='https://docs.google.com/spreadsheets/d/e/2PACX-1vQba1g_agvC0XCOvOoZAQE1R7dPUSDGsOV5wminS5XAOfcVNufGZNkVs4iP5Lt2_rnbU96csHeYK6Ks/pub?output=xlsx'# the link to the data

In [2]:
%%html
<iframe src="https://docs.google.com/spreadsheets/d/1w-dzjSGsZB2mdzRK5dYEDSRfn0eDP1RkYCIB-M1iKZI/edit?usp=sharing" width="700" height="400"></iframe>

2. Open each sheet:

Above you have the link, but not the data yet. Let me get each comparisson sheet:

In [6]:
import pandas as pd

pairwise_learning = pd.read_excel(linkGoogle,sheet_name='learning', index_col=0) # notice
pairwise_friends = pd.read_excel(linkGoogle,sheet_name='friends', index_col=0)
pairwise_school_life = pd.read_excel(linkGoogle,sheet_name='school_life', index_col=0)
pairwise_vocational = pd.read_excel(linkGoogle,sheet_name='vocational_training', index_col=0)
pairwise_college = pd.read_excel(linkGoogle,sheet_name='college_prep', index_col=0)
pairwise_music = pd.read_excel(linkGoogle,sheet_name='music_classes', index_col=0)
pairwise_criteria = pd.read_excel(linkGoogle,sheet_name='criteria', index_col=0)

# The Data Preparation

3. Transform all adjacency matrices into pairwise comparissons:

In [8]:
import networkx as nx
import numpy as np

# learning
G_learning = nx.from_pandas_adjacency(pairwise_learning, create_using=nx.MultiDiGraph())
learning_comparisons ={(e[0],e[1]):e[2]['weight'] for e in G_learning.edges(data=True) if np.isfinite(e[2]['weight'])}

# friends
G_friends = nx.from_pandas_adjacency(pairwise_friends,create_using=nx.MultiDiGraph())
friends_comparisons={(e[0],e[1]):e[2]['weight'] for e in G_friends.edges(data=True) if np.isfinite(e[2]['weight'])}

# school life
G_school_life = nx.from_pandas_adjacency(pairwise_school_life,create_using=nx.MultiDiGraph())
school_life_comparisons={(e[0],e[1]):e[2]['weight'] for e in G_school_life.edges(data=True) if np.isfinite(e[2]['weight'])}

# vocational
G_vocational = nx.from_pandas_adjacency(pairwise_vocational,create_using=nx.MultiDiGraph())
vocational_comparisons={(e[0],e[1]):e[2]['weight'] for e in G_vocational.edges(data=True) if np.isfinite(e[2]['weight'])}

# college prep
G_college = nx.from_pandas_adjacency(pairwise_college,create_using=nx.MultiDiGraph())
college_comparisons={(e[0],e[1]):e[2]['weight'] for e in G_college.edges(data=True) if np.isfinite(e[2]['weight'])}

# music classes
G_music = nx.from_pandas_adjacency(pairwise_music,create_using=nx.MultiDiGraph())
music_comparisons={(e[0],e[1]):e[2]['weight'] for e in G_music.edges(data=True) if np.isfinite(e[2]['weight'])}

G_CRIT = nx.from_pandas_adjacency(pairwise_criteria,create_using=nx.MultiDiGraph())
criteria_comparisons ={(e[0],e[1]):e[2]['weight'] for e in G_CRIT.edges(data=True) if np.isfinite(e[2]['weight'])}

# AHP



4. The installation: we will install ahpy, but we also need a numpy version lower than 2.0

In [10]:
## install ONLY if you are in ANACONDA in a 3.10 environment, then restart kernel.
## if you use BINDER, do not install neither
## do NOT use in colab

!pip install --force-reinstall numpy==1.26.4
!pip install ahpy

  Using cached numpy-1.26.4-cp312-cp312-macosx_11_0_arm64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp312-cp312-macosx_11_0_arm64.whl (13.7 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4


Once installed, we can call the library and use the **Compare** function:

In [14]:
import ahpy

learning = ahpy.Compare('learning', learning_comparisons, random_index='saaty')
friends = ahpy.Compare('friends', friends_comparisons, random_index='saaty')
school_life = ahpy.Compare('school_life', school_life_comparisons, random_index='saaty')
vocational = ahpy.Compare('vocational_training', vocational_comparisons, random_index='saaty')
college = ahpy.Compare('college_prep', college_comparisons, random_index='saaty')
music = ahpy.Compare('music_classes', music_comparisons, random_index='saaty')
criteria = ahpy.Compare('criteria', criteria_comparisons, random_index='saaty')

6. Create hierarchy:

Remember that we have the **hierarchy** Alternatives -> Criteria -> Goal.

At this stage, you just need to tell which are the children of the criteria.


In [16]:
criteria.add_children([learning, friends, school_life, vocational, college, music])

The criteria are then weighted, and the alternatives are scored relative to each other based on the decision-maker's performance of a series of pairwise comparisons. This weighting and scoring process generates a total score for each alternative, by which they are ranked.

We can see which criterion was more valuable like this:

In [19]:
print(criteria.global_weights)

{'learning': 0.3208, 'college_prep': 0.2374, 'friends': 0.1395, 'music_classes': 0.1391, 'vocational_training': 0.1285, 'school_life': 0.0348}


In [ ]:
From the above, we can see the specific weights of each criterion that the schools are being evaluated by.

We can see that **Learning** was considered to have the most relative importance to Prof Saaty's son. Then, after Learning, the criteria ranked from most relative importance to least was:

2. College Prep
3. Friends
4. Music Classes
5. Vocational Training
6. School Life

6. See result:

Now, you may know which is the best option:

In [22]:
print(criteria.target_weights)

{'B': 0.3785, 'A': 0.3674, 'C': 0.2542}


From this result, we can see which alternative (school) ranked the highest, which was **B, Haverford**.

However, the difference between the score for B, Haverford (0.3785), and A, Lower Merion (0.3674), is not significantly large. 

As Prof Saaty says in his lecture video, a one-percent difference could be systemic error. 

He even mentioned that his son complained that "Haverford (B) is not much better than Lower Merion (A)", which turned out to be true! 

7. Assess consistency

The AHP algorithm assumes that when you are comparing you are consistent; but it may detect if you have been inconsistent:

In [29]:
[(val.name,val.consistency_ratio) for val in [learning, friends, school_life, vocational, college, music, criteria]]

[('learning', 0.0516),
 ('friends', 0.0),
 ('school_life', 0.0),
 ('vocational_training', 0.2005),
 ('college_prep', 0.0),
 ('music_classes', 0.0516),
 ('criteria', 0.2272)]

We should review comparisons if we get values greater than 0.1. If your consistency ratio is not less than or equal to 0.1, then it is necessary to revise your judgments.

For this instance, **vocational_training** (0.2005) and **criteria** (0.2272) both are above 0.1.

Future steps to reduce these ratios are to:
- Review the pairwise comparison matrix
- Pinpoint the judgments that deviate most significantly from the expected ratios